# Análises da Desigualdade

Esta seção toma como referência dados de população e rendimentos do Censo de 2010 e os utiliza como *inputs* para a biblioteca [inequality](https://pysal.org/inequality/index.html), a fim de não só medir a desigualdade de uma área de estudo, mas de, também, identificar a estrutura espacial dessa desigualdade.

# Procedimentos Preliminares

## Bibliotecas

In [22]:
import os
import pathlib
import re
import requests
from string import ascii_uppercase

import basedosdados as bd
import geobr
import geopandas as gpd
import IPython.display as display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from getpass import getpass
from shapely.geometry import Point, LineString, box
from statsmodels.stats.weightstats import DescrStatsW
from tobler.area_weighted import area_interpolate
from tobler.util import h3fy

## Parâmetros de Base

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('Solarize_Light2')

In [3]:
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
gcloud_id = getpass('Google Cloud Services billing ID:')

Google Cloud Services billing ID: ········


In [27]:
# Forneça código do IBGE da cidade, ou lista de códigos
# se quiser cidades vizinhas. P. ex.:
# ibge_id = 3106200 -> Belo Horizonte
ibge_id = [3300704] # -> Niteroi e Sao Goncalo e Itaboraí

# Das Malhas Territorias

A malha hexagonal H3 será utilizada como recorte espacial principal ao longo das análises†. Recortes hexagonais apresentam vantagens importantes frente a outras geometrias, sobretudo, eles podem reduzir vieses amostrais derivados de efeitos de borda e, adicionalmente, são mais mais adequados do que malhas retangulares para analisar fenômenos em que relações de vizinhança e conectividade são importantes (Birch, Oom, & Beecham, 2007). Essas vantagens da malha hexagonal serão mais imporantes quando estiverem sendo executadas as análises de acessibilidade, mas, até lá, também foi mostrado por Birch et al. (2007) que hexágonos favorecem a visualização de padrões e mitigam, em alguma medida, efeitos de borda.

Esses efeitos de borda, simplificadamente, dizem respeito a erros de representação na variação local dos dados. Isto é, outras geometrias — malhas quadriculadas, p. ex. — são mais propensas a apresentar variações bruscas e artificiais entre vizinhos — uma quadrícula com valores muito altos ladeada por quadrículas com valores muito baixos. Isso prejudica a análise e a visualização, assim favorecendo erros de interpretação.

Dados socioeconômicos e demográficos, por sua vez, serão imputados em cada hexágono a partir do enso demográfico de 2010. Esses dados censitários serão requisitados ao datalake público da Base dos Dados‡, enquanto que as malhas desses setores serão obtidas junto à biblioteca geobr⨥. Assim, de posse das malhas do IBGE e dos respectivos dados, procedimentos de interpolação espacial⁙ farão a transferência dos dados do censo para os hexágonos.

Complementarmente, para além das informações dos setores censitários, a grade estatística⁜ também será utilizada como insumo, a qual apresenta contagens populacionais com uma resolução maior que a dos agregados por setor: há uma malha com resolução de 200x200 m em áreas urbanas e de 1x1 km em áreas rurais. A grade é importante porque remove as áreas inabitadas — a Floresta d Tijuca, no Rio de Janeiro, p. ex. — e porque permitem localizar o centroide dos hexágonos de maneira mais precisa: seguindo recomendação de Stępniak and Jacobs-Crisioni (2017), o centro de cada geometria deve ser ponderado pela distribuição populacional nela contida, a fim de que haja uma representação mais precisa dos tempos de viagem entre geometrias.

Resta definir qual a resolução dessas malha. Conforme demonstrado, por exemplo, em Pereira et al. (2019), é importante avaliar distintos tamanhos de geometria, a fim de verificar se os resultados obtidos são realmente robustos ou se não passam de artefatos computacionais. Mas, quais resoluções⁕? A menor escala na qual ainda há dados disponíveis é a grade estatística, de maneira que hexágonos menores que ela não agregariam nenhum valor à análise, ao mesmo tempo em que elevariam, desnecessariamente, o tempo de processamento computacional. Com efeito, a menor resolução utilizada é a 9, com área relativamente próxima a um quadrado de lado 200m. No outro extremo, resta definir a maior resolução utilizada. A maior delas será a 7, com área de aproximadamente 5 km2. Tamahos maiores agregariam grandes regiões muito distintas entre si, contribuindo em demasiado para uma ilusão de homogeneidade a qual não existe — p. ex., a resolução 6 já conta com área de cercade 35km2. Portanto, serão avaliadas as resoluções 7, 8 e 9.

    Birch, C. P., Oom, S. P., & Beecham, J. A. (2007). Rectangular and hexagonal grids used for observation, experiment and simulation in ecology. Ecological modelling, 206(3-4), 347-359.
    
    Pereira, R. H., Banister, D., Schwanen, T., & Wessel, N. (2019). Distributional effects of transport policies on inequalities in access to opportunities in Rio de Janeiro. Journal of Transport and Land Use, 12(1), 741-764.
    
    Stępniak, M., & Jacobs-Crisioni, C. (2017). Reducing the uncertainty induced by spatial aggregation in accessibility and spatial interaction applications. Journal of Transport Geography, 61, 17-29.

    † Especificamente, será feito uso das bibliotecas em Python implementadas pela Uber (https://h3geo.org/), baseadas no sistema proposto em:

        Sahr, K., White, D., & Kimerling, A. J. (2003). Geodesic discrete global grid systems. Cartography and Geographic Information Science, 30(2), 121-134.
        
    ‡ ver https://basedosdados.org/

    ⨥ ver https://github.com/ipeaGIT/geobr
    
    ⁙ https://github.com/pysal/tobler

    ⁜ obtida diretamente do site do IBGE em https://censo2010.ibge.gov.br/component/saladeimprensa/?busca=1&id=3&idnoticia=3123&t=grade-estatistica-permite-obter-dados-censo-2010-diversos-recortes-espaciais&view=noticia
    
    ⁕ ver https://h3geo.org/docs/core-library/restable para uma lista de resoluções e as respectivas dimensões

## Malhas Censitárias

In [28]:
def get_study_area(ibge_id):
    if not isinstance(ibge_id, list):
        ibge_id = [ibge_id]

    return pd.concat([
        geobr.read_census_tract(id_)
        for id_
        in ibge_id
        ])


tracts = get_study_area(ibge_id)

In [29]:
crs = tracts.estimate_utm_crs(datum_name='SIRGAS 2000')
crs

<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [30]:
tracts = tracts.to_crs(crs)

Abaixo, utiliza-se a função h3fy, cuja documentação pode ser encontrada [aqui](https://pysal.org/tobler/generated/tobler.util.h3fy.html).

In [31]:
hexes = h3fy(tracts, resolution=8)

# A Leitura dos Atributos Territoriais

Resgatando o que foi dito no preâmbulo deste script, estamos interessados nas seguintes variáveis:
1. Contagens populacionais
2. Contagem de domicílios
3. Rendimentos médios
4. Contagens de postos de trabalho
5. Índice de bem estar urbano

A ideia, portanto, é fazer a leitura e o ajuste dessas variáveis para que possam ser imptuadas nas malhas hexagonais H3.

## Rendimentos Médios

Os rendimentos serão obtidos a partir da API da Base dos Dados. Especificamente, utiliza-se a seguinte variável da tabela básica do Censo de 2010:

- v009: Valor do rendimento nominal médio mensal das pessoas de 10 anos ou mais de idade (com e sem rendimento)

In [32]:
def _sql_style_list(sequence):
    single_quoted_elements = [f"'{c}'" for c in sequence]
    return ', '.join(single_quoted_elements)

In [33]:
def read_census_data(query, gcloud_id):
    data = bd.read_sql(query,
                       billing_project_id=gcloud_id)
    
    return (
        data.astype({'id_setor_censitario': np.int64})
            .astype({'id_setor_censitario': str})
            .rename(
                columns={
                    'v009': 'rendimento_medio',
                    'v002': 'habitantes',
                    'v001': 'domicilios',
                    }
                )
        )

In [34]:
query = (
    'SELECT id_setor_censitario, v009, v002, v001 '
    'FROM basedosdados.br_ibge_censo_demografico.setor_censitario_basico_2010 '
    f"WHERE id_setor_censitario IN ({_sql_style_list(tracts.code_tract.unique())})"
    )
tract_data = read_census_data(query, gcloud_id)

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


In [35]:
tract_data.head(3)

,id_setor_censitario,rendimento_medio,habitantes,domicilios
0,330070405000330,"1,573.84",76.00,38.00
1,330070405000337,730.00,4.00,2.00
2,330070405000022,"1,895.47",78.00,39.00


## Imputacao: Dados Censitários -> Malhas Censitarias

Esta parte faz a junçao dos dados dos setores censitários com as respectivas geometrias dos setores. Após esse processo, será feita a interpolação espacial dos dados para os Hexágonos H3.

In [36]:
# União entre malha de setores e respectivos dados
tracts_with_data = (
    tracts
    .merge(
        tract_data,
        left_on='code_tract',
        right_on='id_setor_censitario',
        how='left',
    )
    .reindex(
        columns=[
            'id_setor_censitario',
            'code_muni',
            'name_muni',
            'habitantes',
            'domicilios',
            'rendimento_medio',
            'geometry',
        ]
    )
)

tracts_with_data.head()

,id_setor_censitario,code_muni,name_muni,habitantes,domicilios,rendimento_medio,geometry
0,330070405000262,"3,300,704.00",Cabo Frio,535.00,159.00,680.07,"POLYGON ((801508.79 7462577.321, 801426.751 74..."
1,330070405000263,"3,300,704.00",Cabo Frio,439.00,147.00,609.65,"POLYGON ((801384.232 7462560.12, 801354.682 74..."
2,330070405000264,"3,300,704.00",Cabo Frio,585.00,196.00,778.69,"POLYGON ((803880.773 7466127.806, 803784.262 7..."
3,330070405000265,"3,300,704.00",Cabo Frio,214.00,79.00,"2,328.87","POLYGON ((806962.997 7465990.925, 806905.973 7..."
4,330070405000266,"3,300,704.00",Cabo Frio,206.00,56.00,97.34,"POLYGON ((806558.95 7466488.358, 806508.976 74..."


## Interpolações: Setor Censitário → Hexágonos

In [37]:
hex_with_data = (
    area_interpolate(
        source_df=tracts_with_data.assign(
            geometry=lambda x: x.buffer(0)
            ),
        target_df=hexes,
        extensive_variables=['habitantes', 'domicilios'],
        intensive_variables=['rendimento_medio',]
        )
    .drop(columns='geometry')
    .set_index(hexes.index)
    .pipe(
        lambda df: pd.merge(
            hexes,
            df,
            left_index=True,
            right_index=True,
            )
        )
    )

In [38]:
hex_with_data.head(2)

,geometry,habitantes,domicilios,rendimento_medio
hex_id,,,,
88a8a2454bfffff,"POLYGON ((804974.786 7491404.522, 804890.376 7...",36.67,11.20,558.48
88a8a24213fffff,"POLYGON ((808653.661 7480743.723, 808218.824 7...",10.59,3.08,493.39


A visualização do mapa abaixo é feita com o método GeoDataFrame.explore do GeoPandas. Você pode [consultar a documentação](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html) para entender melhor como o método funciona.

In [39]:
def classify_income_deciles(
        df,
        income_col='rendimento_medio',
        weight_col='habitantes',
        ):
    """
    Classifies each row into population-weighted income deciles using statsmodels.
    
    Parameters:
        df: pd.DataFrame
        income_col: str - column name with income
        weight_col: str - column name with population weights
    
    Returns:
        pd.Series with decile labels (1–10)
    """
    d = DescrStatsW(df[income_col], weights=df[weight_col])
    # Compute the quantile breakpoints for deciles
    quantiles = [d.quantile(q) for q in np.linspace(0.1, 1.0, 10)]
    bins = [df[income_col].min() - 1] + [float(q) for q in quantiles]
    
    # Assign decile labels 1–10
    return pd.cut(df[income_col], bins=bins, labels=range(1, 11), include_lowest=True)

hex_with_data['decil_renda'] = classify_income_deciles(hex_with_data)


In [41]:
hex_with_data.explore(column='decil_renda', cmap='RdBu', tiles='Carto DB positron')

O mapa acima mostra a distribuição dos domicílios nos municípios de Niterói, São Gonçalo e Itaboraí. Uma coisa que talvez seja perceptível é que, na criação da escala de cores do mapa, há algumas pequenas regiões com cores muito intensas, enquanto a maior parte parece homogênea.

Isso acontece porquê ao representar dados como a quantidade de domicílios por bairro em uma cidade, é comum encontrar distribuições muito assimétricas — ou seja, a maior parte dos bairros tem poucos domicílios, enquanto poucos bairros concentram números muito altos. Assim, se utilizarmos uma escala uniforme, como de 0 a 3000 domicílios, corremos o risco de comprometer a leitura do mapa.

Nesse tipo de situação, a maioria dos bairros ficará representada com cores muito semelhantes, próximas ao início da escala, e apenas os bairros com maior número de domicílios se destacarão visualmente. Isso esconde padrões importantes e dificulta a análise espacial, já que pequenas variações entre os bairros mais comuns passam despercebidas.

Para tornar o mapa mais informativo, é recomendável utilizar métodos de classificação que redistribuam melhor os valores, como a divisão por quantis ou o método de Jenks, ou até mesmo aplicar transformações nos dados (como logaritmos). Assim, é possível valorizar visualmente as diferenças reais entre os bairros, mesmo que os valores absolutos estejam concentrados em uma faixa pequena da distribuição.

# Exercício: Mapas Coropléticos

Objetivo: Compreender os princípios básicos do mapeamento coroplético e aplicar técnicas de classificação e escolha de cores para representar dados espaciais.

Contexto:

Mapas coropléticos são representações cartográficas que utilizam variações de cor para mostrar valores estatísticos agregados por regiões (como estados, municípios ou bairros). Cada região é associada a um valor e preenchida com uma cor que corresponde à sua classe de valor. Embora hoje seja possível criar mapas sem classificação (unclassed), a abordagem com classes (classed) continua sendo valiosa por facilitar a interpretação visual dos dados.

Três decisões fundamentais no mapeamento coroplético:
- Número de classes: Definir em quantos grupos os valores serão divididos.
- Método de classificação: Escolher o algoritmo para agrupar os dados (ex: quantis, intervalos iguais, Jenks).
- Escolha das cores: Aplicar uma paleta que represente adequadamente as diferenças entre os grupos.

Você deve explorar esses conceitos e, para isso, irá utilizar bibliotecas Python como geopandas e [pacotes do conjunto de bibliotecas PySAL](https://pysal.org/). Acompanhe o capítulo Choropleth Mapping do livro [Geographic Data Science with Python](https://geographicdata.science/book/notebooks/05_choropleth.html) e reproduza o exercício prático lá contido com os municípios acima. Documente o passo a passo e discuta e analise seus achados.